In [7]:
import numpy as np

In [2]:
def correct_header(header):
    affine = np.zeros((3, 3), dtype="float")
    affine[0:3, 0:3] = -header["space directions"]
    affine[2, 2] = -affine[2, 2]

    header["space directions"] = np.copy(affine)

    affine = -header["space origin"]
    affine[2] = -affine[2]

    header["space origin"] = np.copy(affine)


In [15]:
from skimage.filters import frangi
import nrrd
import numpy as np


def save_frangi(filename: str, target_filename: str, sigmas=[0.5, 0.7, 0.9, 1.1, 1.3]):
    data, header = nrrd.read(filename)
    inverted_data = np.max(data) - data - 1024
    
    filtered_image = frangi(inverted_data, sigmas=sigmas)
    filtered_image = filtered_image * 1000 / np.max(filtered_image)
    
    nrrd.write(target_filename, filtered_image, header)


In [4]:
import nibabel as nib
import nrrd
import numpy as np


def nrrd_to_nifti(filename, target_filename):
    data, header = nrrd.read(filename)

    affine = np.zeros((4, 4), dtype="float")
    affine[0:3, 0:3] = -header["space directions"]
    affine[3, 3] = 1
    affine[2, 2] = -affine[2, 2]

    affine[0:3, 3] = -header["space origin"]
    affine[2, 3] = -affine[2, 3]

    img = nib.Nifti1Image(data, affine)

    nib.save(img, target_filename)

    return affine


In [5]:
import os
from scancovia import AiSegment


def save_lungs_seg(filename: str, target_filename: str):

    _, header = nrrd.read(filename)

    nrrd_to_nifti(filename, os.path.splitext(filename)[0] + ".nii.gz")
    filename = filename.replace(".nrrd", ".nii.gz")

    ai_segment = AiSegment()
    output = ai_segment(filename)

    nrrd.write(target_filename, output["lungs_mask"].transpose(1, 0, 2), header)

    os.remove(filename)

    return output


In [8]:
import nrrd

def nrrd_get_data(filename):
    data, _ = nrrd.read(filename)
    return data

In [9]:
import numpy as np

def get_emboly_coverage(emboly_seg_path: str, vessels_seg_path: str, threshold=None):

    ground_truth = np.where(nrrd_get_data(emboly_seg_path) > 0, 1, 0)

    vessels_seg = nrrd_get_data(vessels_seg_path)

    if threshold is not None:
        vessels_seg = np.where(vessels_seg > threshold, 1, 0)

    return np.sum(ground_truth * vessels_seg) / np.sum(ground_truth)


In [14]:
import numpy as np

def get_recall(true_seg_path: str, vessels_seg_path: str, threshold=None):

    ground_truth = np.where(nrrd_get_data(true_seg_path) > 0, 1, 0)

    vessels_seg = nrrd_get_data(vessels_seg_path)

    if threshold is not None:
        vessels_seg = np.where(vessels_seg > threshold, 1, 0)
    else:
        vessels_seg = np.where(vessels_seg > 0, 1, 0)

    return np.sum(ground_truth * vessels_seg) / np.sum(ground_truth)


In [19]:
threshold = 1

print(get_recall("../data/CAD_PE/rs/0005RefStd.nrrd", "./data/005_frangi.nrrd", threshold=threshold))
print(get_recall("./data/005_lungs.nrrd", "./data/005_frangi.nrrd", threshold=threshold))

0.22693959640032724
0.08963314980475266


In [17]:
get_recall("./data/005_lungs.nrrd", "./data/005_frangi.nrrd", threshold=0.01)

0.21793437974779326

In [16]:
save_frangi("../data/CAD_PE/images/005.nrrd", "./005_frangi.nrrd")

In [17]:
output = save_lungs_seg("../data/CAD_PE/images/005.nrrd", "./005_lungs.seg.nrrd")

In [7]:
data, header = nrrd.read("./data/005_frangi.nrrd")

In [60]:
nrrd.write("010_lung_mask.seg.nrrd", output["lungs_mask"].transpose(1,0,2), header)

In [39]:
img = nib.Nifti1Image(output["lungs_mask"], affine)

nib.save(img, "010_lungs_mask.seg.nii")

In [15]:
from scancovia import AiSegment

ai_segment = AiSegment() 
output = ai_segment('010.nii')

In [20]:
data.shape

(512, 512, 426)

In [30]:
np.max(output["lungs_mask"])

2

In [37]:
affine = nrrd_to_nifti("../data/CAD_PE/images/010.nrrd", "./010.nii")